In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Etiquetado de beneficiarios a partir del histórico de estado nutricional (peso/talla)**

####**Descripción:**
Clasifica los beneficiarios de acuerdo a:
- si presentaron o no desnutrición (z-score peso/talla $<-2$) en algún momento del estudio (`desnutricion`);
- si se recuperaron de esta condición (`recuperacion`);
- si recayeron en esta condición después de recuperarse (`reincidencia`).

Genera una tabla de relación entre los identificadores `IdBeneficiario` y las etiquetas booleanas de `desnutricion`, `recuperacion` y `reincidencia` y cruza esta tabla con la tabla de datos sociodemográficos con información de cada beneficiario.
####**Requerimientos:** 
-  Diccionario `datatypes_dictionaries/dtypes_var_toma.json`, que mapea cada columna de la tabla de datos nutricionales al *datatype* de Pandas más adecuado.
-  Diccionario `datatypes_dictionaries/dtypes_sociodemo.json`, que mapea cada columna de la tabla de datos sociodemográficos al *datatype* de *Pandas* más adecuado.
- Diccionario `datatypes_dictionaries/cols_ord.json` que incluye una lista de las variables categóricas ordinales de cada tabla de datos.
- Tabla de datos nutricionales limpia `clean_data/tomas_nutricionales.parquet`.
- Tabla de datos sociodemográficos de cada beneficiario preprocesada para modelado `clean_data/sociodemo_model.parquet`.
 
####**Salidas:**
- Tabla de relación entre los identificadores `IdBeneficiario` y las etiquetas `clean_data/IdBeneficiario_labels.parquet`.
- Conjuntos serializados `auxiliary_data/IdBeneficiario_desnutricion.pickle`, `auxiliary_data/IdBeneficiario_recuperacion.pickle` y `auxiliary_data/IdBeneficiario_reincidencia.pickle` de los identificadores `IdBeneficiario` de los beneficiarios que presentaron al menos una vez la condición de desnutrición, recuperación o reincidencia, respectivamente.
- Tabla de datos `clean_data/sociodemo_beneficiario_labels.parquet` con los datos sociodemográficos y las etiquetas de cada beneficiario.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import re
import joblib

### **Lectura de archivos**

In [ ]:
t0 = time.time()
# Abre el diccionario con la estructura de datos definida para ambas tablas de datos
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Lee las tablas de datos limpias
tom = pd.read_parquet('clean_data/tomas_nutricionales.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()
soc = pd.read_parquet('clean_data/sociodemo_model.parquet')
soc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19084096 entries, 0 to 19084095
Data columns (total 33 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float32       
 9   Peso                            float32       
 10  Talla                           float32       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

### **Preliminares**

In [ ]:
# Elimina las tomas nutricionales que no tienen edad asignada
tom = tom[tom['EdadDias'].notnull()]
# Ordena los registros por 'IdBeneficiario' y por 'EdadDias' de modo que 
# para cada beneficiario los registros se ordenan cronológicamente
tom = tom.sort_values(['IdBeneficiario', 'EdadDias']).reset_index(drop=True)

### **Codificación del histórico de las categorías de `EstadoPesoTalla`**
Codifica el estado peso/talla en un número natural (en formato de texto) según el orden de los z-scores: desde `Desnutrición aguda severa` $\rightarrow$ `0`, hasta `Obesidad` $\rightarrow$ `6`. 

Como los valores nulos se codifican en `-1`, se extrae el primer caracter, para codificarlo como `-`.

Posteriormente, se concatenan todas los códigos en orden cronológico para cada beneficiario.

In [ ]:
# Codifica cada categoría como número natural en formato de texto y extrae el primer caracter
tom['CodigoPesoTalla'] = tom['EstadoPesoTalla'].cat.codes.astype('string').str[0]
# Concatena los códigos de categoría de cada variable para cada beneficiario
his = tom.groupby('IdBeneficiario')['CodigoPesoTalla'].apply(lambda x: ''.join(x)).reset_index()
print(f'Historia del estado peso/talla codificada: {time.time() - t0:.2f} s.')

Historia del estado peso/talla codificada: 240.16 s.


### **Etiquetado de los beneficiarios**
- **Desnutrición**: Al menos un evento de desnutrición, según el criterio estricto sugerido por el ICBF: z-score peso/talla $<-2$. Se identifica por categorías `0` (*Desnutrición aguda severa*) y `1` (*Desnutrición aguda moderada*) del `EstadoPesoTalla`.
- **Recuperación**: Al menos un evento de no-desnutrición posterior a al menos un evento de desnutrición.
- **Reincidencia**: Al menos un evento de desnutrición posterior a una recuperación.

In [ ]:
# Patrones de expresiones regulares para cada condición
patron = {
  'desnutricion': '.*[0-1]+.*',
  'recuperacion': '.*[0-1]+.*[2-6]+.*',
  'reincidencia': '.*[0-1]+.*[2-6]+.*[0-1]+.*'
  }
# Define columnas booleanas para marcar cada beneficiario según haya atravesado una situación de
# desnutricion, recuperación o reincidencia en desnutrición
for col in patron.keys():
  his[col] = his['CodigoPesoTalla'].apply(lambda x: bool(re.match(patron[col], x)))
# Elimina la columna del histórico de fechas peso talla
his = his.drop(columns=['CodigoPesoTalla'])
# Guarda el dataframe a un archivo parquet
his.to_parquet('clean_data/IdBeneficiario_labels.parquet')
print(f'Datos etiquetados y guardados: {time.time() - t0:.2f} s.')

Datos etiquetados y guardados: 254.32 s.


### **Conjuntos de beneficiarios en cada condición**

In [ ]:
for condicion in ['desnutricion', 'recuperacion', 'reincidencia']:
  IdB_condicion = set(his.loc[his[condicion] == True, 'IdBeneficiario'])
  print(f"{len(IdB_condicion)} beneficiarios en condición de '{condicion}'.")
  joblib.dump(IdB_condicion, f'auxiliary_data/IdBeneficiario_sets/IdBeneficiario_{condicion}.pickle')
print(f'Identificadores guardados para cada condición: {time.time() - t0:.2f} s.')

178165 beneficiarios en condición de 'desnutricion'.
151866 beneficiarios en condición de 'recuperacion'.
16162 beneficiarios en condición de 'reincidencia'.
Identificadores guardados para cada condición: 255.35 s.


### **Comparación entre Dirección de Primera Infancia y Dirección de Nutrición**
Como es de esperarse la Dirección de Nutrición atiende un porcentaje mayor de casos de desnutrición que la Dirección de Primera Infancia.

In [ ]:
tom['EstadoPesoTalla'].groupby(tom['Direccion']).value_counts(normalize=True, sort=False)

Direccion         EstadoPesoTalla             
Primera Infancia  Desnutrición aguda severa       0.002907
                  Desnutrición aguda moderada     0.009137
                  Riesgo de desnutrición aguda    0.056099
                  Peso adecuado para la talla     0.699553
                  Riesgo de sobrepeso             0.174994
                  Sobrepeso                       0.044327
                  Obesidad                        0.012981
Nutrición         Desnutrición aguda severa       0.024092
                  Desnutrición aguda moderada     0.059145
                  Riesgo de desnutrición aguda    0.213678
                  Peso adecuado para la talla     0.611175
                  Riesgo de sobrepeso             0.066960
                  Sobrepeso                       0.019270
                  Obesidad                        0.005679
Name: EstadoPesoTalla, dtype: float64

### **Cruce de datos con tabla de datos sociodemográficos**

In [ ]:
# Cruza los datos sociodemográficos con las etiquetas
join = soc.merge(his, how='inner', on='IdBeneficiario')
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1476536 entries, 0 to 1476535
Columns: 143 entries, Ind_nivel_sisben_4 to reincidencia
dtypes: UInt32(2), UInt8(48), bool(3), float32(6), float64(2), int64(1), int8(1), uint8(80)
memory usage: 346.4 MB


In [ ]:
join[['desnutricion', 'recuperacion', 'reincidencia']].apply(sum, axis='index')

desnutricion    81159
recuperacion    70779
reincidencia     7503
dtype: int64

### **Guardado**

In [ ]:
join.to_parquet('clean_data/sociodemo_beneficiario_labels.parquet')
print(f'Archivo guardado: {time.time() - t0:.2f} s.')

Archivo guardado: 265.38 s.
